In [1]:
import os
import time
import pandas as pd
import pymysql
import numpy as np
from haversine import haversine
from decimal import Decimal
from tqdm.auto import tqdm

def live_db_conn():
    """Live DB 접속 함수"""
    conn = pymysql.connect(
        host = "db-6j3k3.pub-cdb.ntruss.com", 
        user = "redtable",
        password = "fpemxpdlqmf5491!@#",
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor, 
        db = "redtable2021")
    return conn


def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass = pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 식당 일본어 정보

# RT_RESTAURANT_JA_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=e769bbbf-2920-43d6-8e01-ae7a00dc3357

In [2]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_RESTAURANT_JA_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [3]:
# 라이브DB에서는 불필요한 것들을 분류하기 어렵고, 개발DB에서는 지하철 연결이 안되어있어서 나눠서 작업진행함.
with dev_db_conn() as conn:
    sql = """
    SELECT T5.JP AS 'CTY_NM', T3.SRC_ID AS 'RSTRNT_ID', T4.RSTR_NM AS 'RSTRNT_NM',
      IF(T4.LNNO_ADRES = '', NULL, T4.LNNO_ADRES) AS 'LNM_ADDR',
      IF(T4.RDNMADR = '', NULL, T4.RDNMADR) AS 'RDNMADR_NM',
      IF(T1.RSTR_TELNO = '', NULL, T1.RSTR_TELNO) AS 'RSTRNT_TEL_NO',
      '' AS 'SUBWAYST_NM', '' AS 'SUBWAYST_DSTNC_VALUE', T1.RSTR_LA AS 'RSTRNT_LA', T1.RSTR_LO AS 'RSTRNT_LO'
    FROM redtable01.RSTR T1
    INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
    INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
    INNER JOIN redtable01.RSTR_TRANSLATE T4 ON T3.RSTR_ID = T4.RSTR_ID
    INNER JOIN dictionary.shido T5 ON T1.AREA_NM_1 = T5.SHIDO_NM
    WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
    AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%%？%%'
    AND T4.LANG_NM = 'JP1'
    ORDER BY T3.SRC_ID ASC
    """
    df = pd.read_sql(sql, conn)

In [4]:
with live_db_conn() as conn:
    sql = f"""
    SELECT T1.store_id, T3.name, MIN(T1.distance) AS min_distance
    FROM redtable2021.store_landmark T1
    INNER JOIN redtable2021.landmark_master T2 ON T1.landmark_master_id = T2.id
    INNER JOIN redtable2021.landmark_translate T3 ON T2.id = T3.landmark_master_id
    WHERE T2.`type` = '지하철역' AND T3.lang = 'ja' AND T1.store_id IN {tuple(df["RSTRNT_ID"].tolist())}
    GROUP BY T1.store_id
    """
    subway = pd.read_sql(sql, conn)

In [5]:
set_idx = subway.set_index("store_id")["name"].to_dict()
df["SUBWAYST_NM"] = df["RSTRNT_ID"].map(set_idx)

set_idx = subway.set_index("store_id")["min_distance"].to_dict()
df["SUBWAYST_DSTNC_VALUE"] = df["RSTRNT_ID"].map(set_idx)

# 결측치 및 불필요한 데이터 정리
df = df.fillna(np.nan)
df.replace({np.nan:None, '\\n':None}, inplace=True)

In [6]:
df

,CTY_NM,RSTRNT_ID,RSTRNT_NM,LNM_ADDR,RDNMADR_NM,RSTRNT_TEL_NO,SUBWAYST_NM,SUBWAYST_DSTNC_VALUE,RSTRNT_LA,RSTRNT_LO
0,ソウル特別市,10012,ミョンドンジョン,ソウル特別市 中区 忠武路2街 60-3,ソウル特別市 中区 サミルデロ 299,02-3789-5130,ウルチロイック駅,264.35,37.561532,126.98849
1,ソウル特別市,10013,ミョンドン ソソガルビ,ソウル特別市 中区 明洞2街 23,ソウル特別市 中区 ミョンドン10ギル 35-10,None,ウルチロイック駅,185.92,37.562512,126.985993
2,ソウル特別市,10014,鳳雛チムタク,ソウル特別市 中区 忠武路1街 25-8,ソウル特別市 中区 ミョンドン8ナギル 47,02-3789-9381,シチョン駅,304.28,37.561065,126.982289
3,ソウル特別市,10016,明洞焼肉専門店（韓牛房）,ソウル特別市 中区 乙支路2街 199-56,ソウル特別市 中区 ミョンドン9ギル 37-5,02-2267-8132,チョンガク駅,528.49,37.565386,126.983939
4,ソウル特別市,10017,ソルビン,ソウル特別市 西大門区 滄川洞 33-43,ソウル特別市 西大門区 ヨンセロ 23,02-325-8492,シンチョン(新村)駅,351.59,37.557762,126.93669
...,...,...,...,...,...,...,...,...,...,...
871335,釜山広域市,1191094,フェサシクタン,釜山広域市 江西区 智士洞 1172,釜山広域市 江西区 クァハクサンダンロ 502,None,None,None,35.150177,128.81681
871336,大邱広域市,1191095,フレッシュケトリン モトニックテグゴンジャン,大邱広域市 達西区 新塘洞 1095-11,大邱広域市 達西区 タルソデロ 530,053-382-3311,None,None,35.845846,128.493328
871337,釜山広域市,1628785,チャオラン,釜山広域市 海雲台区 中洞 1829,釜山広域市 海雲台区 タルマジギル 30,0507-1356-1253,None,None,35.159937,129.169813
871338,釜山広域市,1628786,YAKITORI 該工,釜山広域市 水営区 民楽洞 24-6,釜山広域市 水営区 ミンラクポンドンロ19ボンギル 30-5,0507-1479-8334,None,None,35.158234,129.127114


In [7]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_RESTAURANT_JA_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df.apply(lambda row:(row["CTY_NM"], row["RSTRNT_ID"], row["RSTRNT_NM"], row["LNM_ADDR"],
                                  row["RDNMADR_NM"], row["RSTRNT_TEL_NO"], row["SUBWAYST_NM"], row["SUBWAYST_DSTNC_VALUE"],
                                  row["RSTRNT_LA"], row["RSTRNT_LO"]), axis=1).tolist()
        curs.executemany(sql, val)